Setup Instructions
==============
These instructions need to be run once to teach Jupyter about the software environment we'd like to use for the tutorial. It's important that you've followed all the [setup steps](https://github.com/cms-jet/JMEDAS/blob/DAS2019/README.md#getting-started-setup), particularly the grid certificate instructions!

The following cell just needs to be executed once. If you logout and log back in, the setup will remain.

In [ ]:
%%bash
#!/bin/bash

CMSSW_VER="CMSSW_10_6_13"
KERNEL_NAME="hats-jec-2021"
DIRFORCHECKOUT="Jets1HATS2021"

set -e
# Get the CMSSW libraries (specifically ROOT)
SCRAM_ARCH=$(ls -d /cvmfs/cms.cern.ch/$(/cvmfs/cms.cern.ch/common/cmsos)*/cms/cmssw/${CMSSW_VER} | tail -n 1 | awk -F / '{ print $4 }')
export SCRAM_ARCH
source /cvmfs/cms.cern.ch/cmsset_default.sh
if [ -d cmssw-env ]; then
    rm -rf cmssw-env
fi
if [ ! -d "$HOME/$DIRFORCHECKOUT" ]; then
    echo "$HOME/$DIRFORCHECKOUT does not exist. Create it"
    mkdir "$HOME/$DIRFORCHECKOUT"
fi
cd "$HOME/$DIRFORCHECKOUT"
scramv1 project -n "${CMSSW_VER}" $CMSSW_VER 

cd "${CMSSW_VER}/src"
eval `scramv1 runtime -sh`
git clone https://github.com/cms-jet/JMEDAS.git Analysis/JMEDAS -b HATS2021
git clone https://github.com/cms-jet/JetToolbox Analysis/JetToolbox -b jetToolbox_102X_v3
scram b -j 4
cd ..

# Make a wrapper script to load CMSSW python
cat << 'EOF' > bin/python_wrapper.sh
#!/bin/bash
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
source /cvmfs/cms.cern.ch/cmsset_default.sh
cd $DIR
eval `scramv1 runtime -sh`
cd -
exec python "$@"
EOF
chmod +x bin/python_wrapper.sh

# Create the kernel
mkdir -p "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME"
cat << EOF > "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME/kernel.json"
{
 "display_name": "$KERNEL_NAME", 
 "language": "python", 
 "argv": [
  "$PWD/bin/python_wrapper.sh", 
  "-m", 
  "ipykernel_launcher", 
  "-f", 
  "{connection_file}"
 ]
}
EOF

# Report OK
echo "Loaded $CMSSW_VERSION into $KERNEL_NAME!"
echo "Checked out setup to $HOME/$DIRFORCHECKOUT/${CMSSW_VER}/src"

Results
=======

If successful, you should see something similar to the following:

```
Loaded CMSSW_10_6_13 into hats-jec-2021!
Checked out setup to YOURHOME/Jets1HATS2021/CMSSW_10_6_13/src
```

The new kernel you just made will then show up in the various Jupyter dropdowns, adding it alongside the stock defaults. If you've just run this notebook, you may need to refresh your Jupyter Home page before the new kernel shows up.